In [6]:
import json

import pandas as pd
import numpy as np

import string

import seaborn as sns
import matplotlib.pyplot as plt

# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import cross_val_score
# from sklearn.cluster import KMeans
# from sklearn.feature_extraction.text import TfidfVectorizer

from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
from nltk.tokenize import word_tokenize


from numpy import array
from numpy import asarray
from numpy import zeros

In [ ]:
df = pd.read_csv('attempt_1.csv')

# CNN on tweets

Still to do: 
- Combine all text data (tweet and description) 

In [5]:
def remove_punctuation(x):
    return ''.join([i for i in x if i not in string.punctuation])

In [28]:
y_array = array([col for col in df['expert']])

In [8]:
corpus = (df['full_text'].apply(remove_punctuation)).to_list()

In [12]:
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(corpus)

In [13]:
vocab_length = len(word_tokenizer.word_index) + 1
vocab_length

56385

In [16]:
embedded_sentences = word_tokenizer.texts_to_sequences(corpus)

In [17]:
length_long_sentence = 280

In [18]:
padded_sentences = pad_sequences(embedded_sentences, length_long_sentence, padding='post')

In [19]:
embeddings_dictionary = dict()
glove_file = open('/Users/ella.franks/Desktop/Projects/Datasets/glove.6B/glove.6B.100d.txt', encoding="utf8")

In [20]:
for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions

glove_file.close()

In [21]:
embedding_matrix = zeros((vocab_length, 100))
for word, index in word_tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [22]:
model = Sequential()
embedding_layer = Embedding(vocab_length, 100, weights=[embedding_matrix], input_length=length_long_sentence, trainable=False)
model.add(embedding_layer)
# model.add(Flatten())
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D())
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [23]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 280, 100)          5638500   
_________________________________________________________________
conv1d (Conv1D)              (None, 276, 128)          64128     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 138, 128)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 134, 128)          82048     
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               16512     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1

In [29]:
model.fit(padded_sentences, y_array, epochs=5, verbose=1, validation_split=0.2)

Epoch 1/5
491/491 [==============================] - 37s 76ms/step - loss: 0.3046 - acc: 0.8779 - val_loss: 0.2183 - val_acc: 0.8843
Epoch 2/5
491/491 [==============================] - 41s 83ms/step - loss: 0.2131 - acc: 0.9225 - val_loss: 0.3960 - val_acc: 0.8328
Epoch 3/5
491/491 [==============================] - 44s 90ms/step - loss: 0.1624 - acc: 0.9441 - val_loss: 0.3647 - val_acc: 0.8517
Epoch 4/5
491/491 [==============================] - 49s 99ms/step - loss: 0.1162 - acc: 0.9603 - val_loss: 0.5043 - val_acc: 0.8445
Epoch 5/5
491/491 [==============================] - 49s 100ms/step - loss: 0.0749 - acc: 0.9743 - val_loss: 0.3780 - val_acc: 0.8942


In [30]:
model.history.history['val_acc']

[0.8843017220497131,
 0.832823634147644,
 0.8516819477081299,
 0.844546377658844,
 0.8942405581474304]

In [35]:
model.save("1st_model")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: 1st_model/assets


# Using saved model to predict on new tweets

In [ ]:
#Get access to new data